In [ ]:
#This binary text classifier is meant to use the trained data to create a classifier to create a dataset of just disclosures!

In [ ]:
## GOOGLE COLAB INSTALL REQUIREMENTS
!pip install transformers
!pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.5 MB/s eta 0:00:00


In [ ]:
### Import Packages ###
import pandas as pd
import torch
import numpy as np
import re
import demoji
from transformers import RobertaTokenizer, RobertaModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
# Install and download necessary packages
demoji.download_codes()

<ipython-input-2-975b9e938a24>:12: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


BUILD CLASSIFIER

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving All Human Coded Disclosure Tweets.csv to All Human Coded Disclosure Tweets.csv


In [ ]:
df = pd.read_csv("All Human Coded Disclosure Tweets.csv")

In [ ]:
##Check dataset head, columns, or length
df.head()
#df.columns
#len(df)

,text,coding (0 or 1),id,username,name,user_description
0,Yup absolutely #vaccinated https://t.co/FsPwBv...,1,1.470764e+18,GoddessTales,Tales from Sanctuary,"😷GardenWitch, Artist, Photographer Bees 🐝 🦋 Bu..."
1,"Your Teacher Rachel got the JAB! /Bakunado ,pr...",1,1.420000e+18,teachRache,RayRach,"tutor, teacher, singlemom, vlogger /streamer ..."
2,Your one your two. \nDefinitely #vaccinated ht...,1,1.420387e+18,Stron9erAnnA,PerlaMiseriah,La mia lingua preferita è il dialetto.\n\n#Fin...
3,"Your new lucky charm bracelet, in your favorit...",0,1.354851e+18,TeamColorsByC,Carrie Dix,Team Spirit Jewelry in your favorite sports te...
4,Your job: Get #vaccinated.\n@gatesfoundation’s...,0,1.386745e+18,AEMarling,Æ Marling,Projects resistance art. SFWA member. he/them/...


PRE PROCESS TEXT

In [ ]:
# Preprocessing function
def preprocess_text(text):
    # Make lowercase
    text = text.lower()
    # Remove punctuation and #
    text = re.sub(r'[#.,!?]', '', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove emojis
    text = demoji.replace(text, '')
    # Remove smileys
    text = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', '', text)
    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    # Replace &amp with "and"
    text = text.replace('&amp', 'and')
    # Replace date content (YYYY-MM-DD format)
    text = re.sub(r'\d{4}-\d{2}-\d{2}', '', text)
    return text.strip()

In [ ]:
# Apply preprocessing to the "text" column and save in "processed_text" column
df['processed_text'] = df['text'].apply(preprocess_text)

In [ ]:
##Check dataset head, columns, or length
df.head()
#len(df)

,coding (0 or 1),processed_text
0,1,yup absolutely vaccinated
1,1,your teacher rachel got the jab /bakunado prot...
2,1,your one your two \ndefinitely vaccinated
3,0,your new lucky charm bracelet in your favorite...
4,0,your job: get vaccinated\n@gatesfoundation’s j...


In [ ]:
# Indicate the columns to keep, Not all are needed
columns_to_keep = ["processed_text", "coding (0 or 1)"] ###CHANGE THIS
# Dropping columns from the dataset
df = df.drop(columns=[col for col in df.columns if col not in columns_to_keep])

In [ ]:
#Rename the column "processed_text" to "text"
df.rename(columns={'processed_text': 'text'}, inplace=True)
#Rename the column of interest to "column_of_interest"
df.rename(columns={'coding (0 or 1)': 'column_of_interest'}, inplace=True) ###CHANGE THIS

In [ ]:
## Delete rows with na
df = df.dropna(how="any", inplace=False)
len(df)

6348

In [ ]:
## Check for and print unique values in a column
unique_values = df["column_of_interest"].unique()
for value in unique_values:
    print(value)

1
0


In [ ]:
# Initialize the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


In [ ]:
### Define the labels mapping ###
labels = {
    0: 0,
    1: 1
}

In [ ]:
### Define a Dataset class to handle your data ###
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['column_of_interest']]
        self.texts = [tokenizer.encode_plus(
            text,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        ) for text in df['text']]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        batch_texts = self.texts[idx]
        batch_y = self.labels[idx]

        input_ids = batch_texts['input_ids'].squeeze(0)
        attention_mask = batch_texts['attention_mask'].squeeze(0)

        return input_ids, attention_mask, batch_y

In [ ]:
### Split the dataframe into training, validation, and test sets (80:10:10 split)###
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8 * len(df)), int(.9 * len(df))])

In [ ]:
print(len(df_train), len(df_val), len(df_test))

5078 635 635


In [ ]:
### Define the RoBERTa classifier model ###
class RoBERTaClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(RoBERTaClassifier, self).__init__()

        self.roberta = RobertaModel.from_pretrained("roberta-base")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
### Train the model ###
def train(model, train_data, val_data, learning_rate, epochs):
    train_dataset = Dataset(train_data)
    val_dataset = Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0

        for input_ids, attention_mask, labels in tqdm(train_dataloader):
            labels = labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            output = model(input_ids, attention_mask)

            batch_loss = criterion(output, labels.long())
            total_loss_train += batch_loss.item()

            acc = (output.argmax(dim=1) == labels).sum().item()
            total_acc_train += acc

            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

        avg_acc_train = total_acc_train / len(train_dataset)
        avg_loss_train = total_loss_train / len(train_dataset)

        print(f"Epoch {epoch_num + 1}/{epochs}")
        print(f"Train Loss: {avg_loss_train:.4f}, Train Accuracy: {avg_acc_train:.4f}")

# Train the model
EPOCHS = 8
model = RoBERTaClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 2539/2539 [01:37<00:00, 26.00it/s]


Epoch 1/8
Train Loss: 0.1960, Train Accuracy: 0.8147


100%|██████████| 2539/2539 [01:34<00:00, 26.91it/s]


Epoch 2/8
Train Loss: 0.1119, Train Accuracy: 0.9143


100%|██████████| 2539/2539 [01:34<00:00, 26.80it/s]


...

Epoch 6/8
Train Loss: 0.0219, Train Accuracy: 0.9894


100%|██████████| 2539/2539 [01:34<00:00, 26.92it/s]


Epoch 7/8
Train Loss: 0.0164, Train Accuracy: 0.9923


100%|██████████| 2539/2539 [01:34<00:00, 26.94it/s]

Epoch 8/8
Train Loss: 0.0112, Train Accuracy: 0.9949


In [ ]:
# After training
torch.save(model.state_dict(), 'roberta_classifier_model.pth')

In [ ]:
### Evaluate Model with Precision, Recall, and F1 score
from sklearn.metrics import classification_report

def evaluate(model, test_data):
    test_dataset = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    predictions = []
    true_labels = []

    with torch.no_grad():
        for input_ids, attention_mask, labels in test_dataloader:
            labels = labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            output = model(input_ids, attention_mask)

            _, predicted_labels = output.max(dim=1)

            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    predicted_labels = np.array(predictions)
    true_labels = np.array(true_labels)

    report = classification_report(true_labels, predicted_labels)
    print(report)

# Evaluate the model
evaluate(model, df_test)

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       420
           1       0.83      0.87      0.85       215

    accuracy                           0.90       635
   macro avg       0.88      0.89      0.89       635
weighted avg       0.90      0.90      0.90       635



In [ ]:
## Evaluate model: print the text instances where the model got it wrong
def evaluate(model, test_data):
    test_dataset = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    predictions = []
    true_labels = []
    texts = []

    with torch.no_grad():
        for input_ids, attention_mask, labels in test_dataloader:
            labels = labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            output = model(input_ids, attention_mask)

            _, predicted_labels = output.max(dim=1)

            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            texts.extend(input_ids)

    report = classification_report(true_labels, predictions)
    print(report)

    # Print instances where true labels and predicted labels are not the same
    for text, true_label, predicted_label in zip(texts, true_labels, predictions):
        if true_label != predicted_label:
            decoded_text = tokenizer.decode(text.squeeze(), skip_special_tokens=True)
            print("Text:", decoded_text)
            print("True Label:", true_label)
            print("Predicted Label:", predicted_label)
            print("=" * 50)

evaluate(model, df_test)


Downloading the Saved Model to Your MacBook Pro:
Once the model is saved, you need to download it from Google Colab to your MacBook Pro. Follow these steps:

a. Run the following code in your Colab notebook to generate a download link for the saved model:

In [ ]:
#from google.colab import files
#files.download('roberta_classifier_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load the trained model
class RoBERTaClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(RoBERTaClassifier, self).__init__()

        self.roberta = RobertaModel.from_pretrained("roberta-base")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

loaded_model = RoBERTaClassifier()
loaded_model.load_state_dict(torch.load('roberta_classifier_model.pth'))
loaded_model.eval()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RoBERTaClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
# Load and preprocess the new dataset
#I uploaded the csv to the files upload option on colab (left side of screen)
new_df = pd.read_csv("RunThroughDisclosureClassifier.csv")

<ipython-input-23-d77045ceb679>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv("RunThroughDisclosureClassifier.csv")


In [ ]:
new_df.head()
#len(new_df)

,author_id,lang,source,possibly_sensitive,created_at,text,id,reply_settings,conversation_id,public_metrics.retweet_count,...,user_created_at,profile_image_url,username,name,verified,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,is_individual
0,1000011480450842624,en,Twitter for Android,False,2021-06-03 03:03:11,Protect your self against disease\n\nVaccinate...,1400286915014709254,everyone,1400286915014709254,1.0,...,2018-05-25 13:51:31,https://pbs.twimg.com/profile_images/113953660...,IgMallikarjun,Mallikarjun IG,False,17,67,32,0,True
1,1000022939893460992,en,Twitter for Android,False,2021-08-05 08:55:26,"Alright gaymers, got the second shot 😎 #vaccin...",1423205999897419782,everyone,1423205999897419782,0.0,...,2018-05-25 14:37:03,https://pbs.twimg.com/profile_images/149393976...,sleepygal_owo,✨♀️Arezu stan♀️✨,False,41,103,1321,0,True
2,1000024947518050305,en,Twitter for Android,False,2021-09-17 07:15:07,Vaccination Dose 2 - 0 Covid. Fully jabbed tod...,1438763429976633345,everyone,1438763429976633345,0.0,...,2018-05-25 14:45:01,https://pbs.twimg.com/profile_images/119023767...,TacticalMaverik,Tactically Obsessed,False,288,163,4834,7,True
3,1000036171676897281,en,Twitter for iPhone,False,2021-08-03 16:39:10,Anyways it’s time to open everything up. Small...,1422597923909427205,everyone,1422597923909427205,0.0,...,2018-05-25 15:29:37,https://pbs.twimg.com/profile_images/139807095...,EdwardLander4,🍁 🇨🇦Edward🇺🇸🦅,False,816,329,25802,0,True
4,1000039629096042497,en,Twitter for iPhone,False,2021-08-14 15:00:45,#vaccinated Got my 1st dose of AstraZ this mor...,1426559425175646211,everyone,1426559425175646211,0.0,...,2018-05-25 15:43:22,https://pbs.twimg.com/profile_images/148197335...,hellojustin18,Jʋƨтίɳ 🌷,False,98,315,1127,0,True


489137

In [ ]:
#preprocess new dataset
new_df['processed_text'] = new_df['text'].apply(preprocess_text)

In [ ]:
# Initialize the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
# Define a function to classify text using the loaded model
def classify_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        logits = loaded_model(inputs['input_ids'], inputs['attention_mask'])
    predicted_label = torch.argmax(logits, dim=1).item()
    return predicted_label

In [ ]:
# Apply the classification function to the new dataset
#new_df['predicted_label'] = new_df['processed_text'].apply(classify_text)

KeyboardInterrupt: ignored

In [ ]:
#Trying to speed things up...
from concurrent.futures import ThreadPoolExecutor
from torch.nn.utils.rnn import pad_sequence

# Define a function to classify a batch of texts
def classify_batch(batch_texts):
    inputs = [tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128) for text in batch_texts]
    batch_input_ids = pad_sequence([input_dict['input_ids'].squeeze(0) for input_dict in inputs], batch_first=True)
    batch_attention_mask = pad_sequence([input_dict['attention_mask'].squeeze(0) for input_dict in inputs], batch_first=True)

    with torch.no_grad():
        logits = loaded_model(batch_input_ids, batch_attention_mask)
    batch_predicted_labels = torch.argmax(logits, dim=1).tolist()

    return batch_predicted_labels


In [ ]:
# Batch processing for classification with optimized tokenization
def batch_classify_text_optimized(texts, batch_size=16):
    num_texts = len(texts)
    predicted_labels = []

    for i in range(0, num_texts, batch_size):
        batch_texts = texts[i:i + batch_size]
        batch_results = classify_batch(batch_texts)
        predicted_labels.extend(batch_results)

    return predicted_labels

In [ ]:
# Batch processing for classification with optimized tokenization and parallel processing
def batch_classify_text_parallel_optimized(texts, batch_size=16):
    num_texts = len(texts)
    predicted_labels = []

    with ThreadPoolExecutor() as executor:
        for i in range(0, num_texts, batch_size):
            batch_texts = texts[i:i + batch_size]
            batch_results = list(executor.map(classify_batch, [batch_texts]))
            predicted_labels.extend(batch_results[0])

    return predicted_labels

In [ ]:
# Apply parallel batch classification with optimized tokenization to the new dataset
new_df['predicted_label'] = batch_classify_text_parallel_optimized(new_df['processed_text'], batch_size=16)

In [ ]:
# Save the output predictions to a CSV file
output_filename = "output_predictions.csv"
new_df.to_csv(output_filename, index=False)
print(f"Predictions saved to {output_filename}")

Predictions saved to output_predictions.csv


In [ ]:
from google.colab import files

# Download the CSV file to your local machine
files.download("output_predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save the output as a CSV file on your desktop
output_csv_path = "/Users/emilymckinley/vaccination_disclosure/NEW TWITTER DATA/A Last effort of perfection/output_predictions.csv"
new_df.to_csv(output_csv_path, index=False)
print("Predictions saved to:", output_csv_path)

UPLOAD DATA AND RUN THROUGH CLASSIFIER